# Spotify Music Recommendation System Based on Mood

This notebook implements a music recommendation system using the Spotify API. We'll:
1. Set up Spotify API authentication
2. Collect music data based on different moods
3. Process and analyze the data
4. Create a recommendation system

In [5]:
# Import required libraries
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors

In [2]:
# Set up Spotify API credentials
client_id = '5fddec0014f8450399c7cc49399fd5d1'
client_secret = 'e15b651655af4c39b045fc1aeb91f7cd'

# Initialize Spotify client
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [23]:
# Define mood-based search queries with MULTIPLE LANGUAGES
# Each language has specific search terms for each mood

language_mood_queries = {
    'English': {
        'Happy': ['happy english', 'upbeat pop', 'cheerful songs', 'joyful music'],
        'Sad': ['sad english', 'heartbreak songs', 'emotional ballad', 'melancholy'],
        'Energetic': ['workout music', 'pump up', 'intense rock', 'power anthem'],
        'Calm': ['relaxing music', 'peaceful ambient', 'meditation', 'chill lofi'],
        'Romantic': ['romantic english', 'love songs', 'passionate ballad', 'valentine'],
        'Motivated': ['motivational', 'inspirational', 'success anthem', 'hustle music'],
        'Party': ['party music', 'dance pop', 'club hits', 'edm'],
        'Focus': ['focus music', 'study lofi', 'concentration', 'productive beats']
    },
    'Hindi': {
        'Happy': ['khushi hindi', 'happy bollywood', 'cheerful hindi songs', 'मस्ती गाने'],
        'Sad': ['sad hindi', 'dard bhare gaane', 'breakup hindi', 'दर्द गाने'],
        'Energetic': ['energetic bollywood', 'dance hindi', 'peppy bollywood', 'जोश गाने'],
        'Calm': ['peaceful hindi', 'sufi songs', 'meditation hindi', 'शांत संगीत'],
        'Romantic': ['romantic hindi', 'pyar ke gaane', 'love bollywood', 'रोमांटिक गाने'],
        'Motivated': ['motivational hindi', 'inspirational bollywood', 'josh songs', 'प्रेरणा गीत'],
        'Party': ['party bollywood', 'dance hindi', 'celebration songs', 'पार्टी गाने'],
        'Focus': ['lofi hindi', 'instrumental bollywood', 'calm hindi', 'पढ़ाई संगीत']
    },
    'Telugu': {
        'Happy': ['happy telugu', 'cheerful tollywood', 'సంతోష పాటలు', 'kuthu telugu'],
        'Sad': ['sad telugu', 'emotional tollywood', 'విచారం పాటలు', 'breakup telugu'],
        'Energetic': ['energetic telugu', 'mass songs', 'పవర్ పాటలు', 'dance telugu'],
        'Calm': ['peaceful telugu', 'devotional songs', 'శాంతి సంగీతం', 'meditation telugu'],
        'Romantic': ['love telugu', 'romantic tollywood', 'ప్రేమ పాటలు', 'melody telugu'],
        'Motivated': ['motivational telugu', 'inspirational tollywood', 'ప్రేరణ గీతాలు', 'success telugu'],
        'Party': ['party telugu', 'item songs', 'పార్టీ పాటలు', 'dance tollywood'],
        'Focus': ['instrumental telugu', 'bgm telugu', 'focus music telugu', 'అధ్యయనం సంగీతం']
    },
    'Tamil': {
        'Happy': ['happy tamil', 'kuthu songs', 'மகிழ்ச்சி பாடல்கள்', 'cheerful kollywood'],
        'Sad': ['sad tamil', 'emotional kollywood', 'துக்க பாடல்கள்', 'breakup tamil'],
        'Energetic': ['energetic tamil', 'mass kollywood', 'சக்தி பாடல்கள்', 'kuthu dance'],
        'Calm': ['peaceful tamil', 'devotional kollywood', 'அமைதி இசை', 'carnatic'],
        'Romantic': ['love tamil', 'romantic kollywood', 'காதல் பாடல்கள்', 'melody tamil'],
        'Motivated': ['motivational tamil', 'inspirational kollywood', 'ஊக்க பாடல்கள்', 'success tamil'],
        'Party': ['party tamil', 'kuthu celebration', 'விழா பாடல்கள்', 'dance kollywood'],
        'Focus': ['instrumental tamil', 'ilayaraja bgm', 'படிப்பு இசை', 'ar rahman instrumental']
    },
    'Malayalam': {
        'Happy': ['happy malayalam', 'cheerful mollywood', 'സന്തോഷ ഗാനങ്ങൾ', 'feel good malayalam'],
        'Sad': ['sad malayalam', 'emotional mollywood', 'വിഷാദ ഗാനങ്ങൾ', 'breakup malayalam'],
        'Energetic': ['energetic malayalam', 'mass mollywood', 'ശക്തി പാട്ടുകൾ', 'dance malayalam'],
        'Calm': ['peaceful malayalam', 'devotional songs', 'ശാന്തി സംഗീതം', 'meditation malayalam'],
        'Romantic': ['love malayalam', 'romantic mollywood', 'പ്രണയ ഗാനങ്ങൾ', 'melody malayalam'],
        'Motivated': ['motivational malayalam', 'inspirational mollywood', 'പ്രചോദന ഗാനങ്ങൾ', 'success malayalam'],
        'Party': ['party malayalam', 'celebration mollywood', 'ആഘോഷ ഗാനങ്ങൾ', 'dance malayalam'],
        'Focus': ['instrumental malayalam', 'bgm mollywood', 'പഠന സംഗീതം', 'peaceful instrumental']
    },
    'Kannada': {
        'Happy': ['happy kannada', 'cheerful sandalwood', 'ಸಂತೋಷ ಹಾಡುಗಳು', 'kuthu kannada'],
        'Sad': ['sad kannada', 'emotional sandalwood', 'ದುಃಖದ ಹಾಡುಗಳು', 'breakup kannada'],
        'Energetic': ['energetic kannada', 'mass sandalwood', 'ಶಕ್ತಿ ಹಾಡುಗಳು', 'dance kannada'],
        'Calm': ['peaceful kannada', 'devotional songs', 'ಶಾಂತಿ ಸಂಗೀತ', 'meditation kannada'],
        'Romantic': ['love kannada', 'romantic sandalwood', 'ಪ್ರೀತಿ ಹಾಡುಗಳು', 'melody kannada'],
        'Motivated': ['motivational kannada', 'inspirational sandalwood', 'ಪ್ರೇರಕ ಹಾಡುಗಳು', 'success kannada'],
        'Party': ['party kannada', 'celebration sandalwood', 'ಪಾರ್ಟಿ ಹಾಡುಗಳು', 'dance sandalwood'],
        'Focus': ['instrumental kannada', 'bgm kannada', 'ಅಧ್ಯಯನ ಸಂಗೀತ', 'hamsalekha instrumental']
    }
}

def collect_tracks_by_mood_and_language(mood, language, queries, tracks_per_query=10):
    """Collect tracks for a specific mood and language using search queries"""
    all_tracks = []
    
    for query in queries:
        try:
            # Search for tracks
            results = sp.search(q=query, type='track', limit=tracks_per_query)
            
            for track in results['tracks']['items']:
                if track and track['id']:
                    track_info = {
                        'id': track['id'],
                        'name': track['name'],
                        'artist': track['artists'][0]['name'] if track['artists'] else 'Unknown',
                        'album': track['album']['name'] if track['album'] else 'Unknown',
                        'popularity': track['popularity'],
                        'duration_ms': track['duration_ms'],
                        'release_date': track['album']['release_date'] if track['album'] else None,
                        'mood': mood,
                        'language': language,
                        'search_query': query
                    }
                    all_tracks.append(track_info)
        except Exception as e:
            print(f"  ⚠ Error with query '{query}': {str(e)}")
            continue
    
    return pd.DataFrame(all_tracks)

print("✓ Setup complete!")
print(f"  Languages: {len(language_mood_queries)}")
print(f"  Moods per language: {len(list(language_mood_queries.values())[0])}")
print(f"  Total combinations: {len(language_mood_queries) * len(list(language_mood_queries.values())[0])}")

✓ Setup complete!
  Languages: 6
  Moods per language: 8
  Total combinations: 48


In [24]:
# Collect tracks for ALL moods and ALL languages
all_data_frames = []

print("🎵 Starting comprehensive data collection...")
print("=" * 60)

for language, moods_dict in language_mood_queries.items():
    print(f"\n📚 Language: {language}")
    print("-" * 60)
    
    for mood, queries in moods_dict.items():
        print(f"  🎭 {mood:15} - Collecting...", end=" ")
        
        # Collect tracks for this mood-language combination
        df = collect_tracks_by_mood_and_language(mood, language, queries, tracks_per_query=8)
        
        if not df.empty:
            all_data_frames.append(df)
            print(f"✓ {len(df)} tracks")
        else:
            print("✗ No tracks found")

# Combine all dataframes
if all_data_frames:
    full_dataset = pd.concat(all_data_frames, ignore_index=True)
    
    # Remove duplicates based on track ID
    initial_count = len(full_dataset)
    full_dataset = full_dataset.drop_duplicates(subset=['id'], keep='first')
    final_count = len(full_dataset)
    
    print("\n" + "=" * 60)
    print("📊 COLLECTION SUMMARY")
    print("=" * 60)
    print(f"  Total tracks collected: {initial_count}")
    print(f"  Duplicates removed: {initial_count - final_count}")
    print(f"  Unique tracks: {final_count}")
    print(f"\n📈 Distribution by Language:")
    print(full_dataset['language'].value_counts().to_string())
    print(f"\n🎭 Distribution by Mood:")
    print(full_dataset['mood'].value_counts().to_string())
    
    # Save to CSV
    output_path = '../data/spotify_mood_tracks_multilang.csv'
    full_dataset.to_csv(output_path, index=False)
    print(f"\n💾 Dataset saved to: {output_path}")
    print("=" * 60)
else:
    print("\n⚠ No data collected!")

🎵 Starting comprehensive data collection...

📚 Language: English
------------------------------------------------------------
  🎭 Happy           - Collecting... ✓ 32 tracks
  🎭 Sad             - Collecting... ✓ 32 tracks
  🎭 Energetic       - Collecting... ✓ 32 tracks
  🎭 Calm            - Collecting... ✓ 32 tracks
  🎭 Romantic        - Collecting... ✓ 32 tracks
  🎭 Motivated       - Collecting... ✓ 32 tracks
  🎭 Party           - Collecting... ✓ 32 tracks
  🎭 Focus           - Collecting... ✓ 32 tracks

📚 Language: Hindi
------------------------------------------------------------
  🎭 Happy           - Collecting... ✓ 32 tracks
  🎭 Sad             - Collecting... ✓ 32 tracks
  🎭 Energetic       - Collecting... ✓ 32 tracks
  🎭 Calm            - Collecting... ✓ 32 tracks
  🎭 Romantic        - Collecting... ✓ 32 tracks
  🎭 Motivated       - Collecting... ✓ 32 tracks
  🎭 Party           - Collecting... ✓ 32 tracks
  🎭 Focus           - Collecting... ✓ 32 tracks

📚 Language: Telugu
------

In [20]:
# Collect tracks for all moods (including new ones)
print("Collecting mood-based music data from Spotify...")
print("="*60)

all_mood_tracks = []

for mood, queries in mood_queries.items():
    print(f"\n📀 Collecting {mood} tracks...")
    df = collect_tracks_by_mood(mood, queries, tracks_per_query=10)
    
    if not df.empty:
        all_mood_tracks.append(df)
        print(f"   ✓ Collected {len(df)} tracks")
    else:
        print(f"   ✗ No tracks collected")

# Combine all dataframes
if all_mood_tracks:
    tracks_df = pd.concat(all_mood_tracks, ignore_index=True)
    
    # Remove duplicate tracks
    original_count = len(tracks_df)
    tracks_df = tracks_df.drop_duplicates(subset=['id'], keep='first')
    duplicates_removed = original_count - len(tracks_df)
    
    # Create data directory if it doesn't exist
    import os
    os.makedirs('../data', exist_ok=True)
    
    # Save dataset
    tracks_df.to_csv('../data/spotify_mood_tracks.csv', index=False)
    
    # Display summary
    print("\n" + "="*60)
    print("✓ DATASET SUCCESSFULLY CREATED!")
    print("="*60)
    print(f"Total tracks collected: {original_count}")
    print(f"Duplicates removed: {duplicates_removed}")
    print(f"Final unique tracks: {len(tracks_df)}")
    print(f"\nTracks distribution by mood:")
    print(tracks_df['mood'].value_counts().sort_index())
    print(f"\n📁 Saved to: data/spotify_mood_tracks.csv")
    print("\nSample of collected tracks:")
    display(tracks_df[['name', 'artist', 'mood', 'popularity']].head(20))
else:
    print("\n❌ No tracks were collected. Please check your API credentials.")


📀 Collecting Happy tracks...
   ✓ Collected 50 tracks

📀 Collecting Sad tracks...
   ✓ Collected 50 tracks

📀 Collecting Sad tracks...
   ✓ Collected 50 tracks

📀 Collecting Energetic tracks...
   ✓ Collected 50 tracks

📀 Collecting Energetic tracks...
   ✓ Collected 50 tracks

📀 Collecting Calm tracks...
   ✓ Collected 50 tracks

📀 Collecting Calm tracks...
   ✓ Collected 50 tracks

📀 Collecting Romantic tracks...
   ✓ Collected 50 tracks

📀 Collecting Romantic tracks...
   ✓ Collected 50 tracks

📀 Collecting Motivated tracks...
   ✓ Collected 50 tracks

📀 Collecting Motivated tracks...
   ✓ Collected 50 tracks

📀 Collecting Party tracks...
   ✓ Collected 50 tracks

📀 Collecting Party tracks...
   ✓ Collected 50 tracks

📀 Collecting Focus tracks...
   ✓ Collected 50 tracks

📀 Collecting Focus tracks...
   ✓ Collected 50 tracks

✓ DATASET SUCCESSFULLY CREATED!
Total tracks collected: 400
Duplicates removed: 15
Final unique tracks: 385

Tracks distribution by mood:
mood
Calm         50

,name,artist,mood,popularity
0,Happy Nation,Ace of Base,Happy,74
1,Happy Birthday To You - Classic Version,Happy Birthday Songs,Happy,49
2,Happy Birthday To You Ji,Mimi Teddy,Happy,33
3,Happy B'Day,D Soldierz,Happy,37
4,Happy Birthday,Diljit Dosanjh,Happy,38
5,Baar Baar Din Ye Aaye-Happy Birthday To You - ...,Mohammed Rafi,Happy,34
6,Happy,All Ok,Happy,52
7,Happy New Year,Vidyasagar,Happy,54
8,Happy Diwali,Vishal-Shekhar,Happy,46
9,Aashiq Tera,Sohail Sen,Happy,69


In [21]:
# Create a mood-based recommendation system
class MoodBasedRecommender:
    def __init__(self, tracks_df):
        self.tracks_df = tracks_df
    
    def recommend_by_mood(self, mood, n_recommendations=10):
        """Get top recommendations for a specific mood"""
        mood_tracks = self.tracks_df[self.tracks_df['mood'] == mood]
        
        if len(mood_tracks) == 0:
            return f"No tracks found for mood: {mood}"
        
        # Sort by popularity and get top N
        top_tracks = mood_tracks.nlargest(n_recommendations, 'popularity')
        
        return top_tracks[['name', 'artist', 'album', 'popularity']]
    
    def recommend_similar(self, track_name, n_recommendations=5):
        """Find similar tracks based on a given track name"""
        # Find the track
        track = self.tracks_df[self.tracks_df['name'].str.contains(track_name, case=False, na=False)]
        
        if track.empty:
            return f"Track '{track_name}' not found in database"
        
        # Get the mood
        mood = track.iloc[0]['mood']
        artist = track.iloc[0]['artist']
        
        # Find similar tracks from same mood, preferably same artist
        same_mood = self.tracks_df[self.tracks_df['mood'] == mood]
        
        # Prioritize tracks by same artist
        same_artist = same_mood[same_mood['artist'] == artist]
        other_artists = same_mood[same_mood['artist'] != artist]
        
        # Combine and get top by popularity
        similar = pd.concat([same_artist, other_artists]).nlargest(n_recommendations + 1, 'popularity')
        
        # Exclude the original track
        similar = similar[similar['name'] != track.iloc[0]['name']]
        
        return similar.head(n_recommendations)[['name', 'artist', 'popularity', 'mood']]
    
    def get_mood_stats(self):
        """Get statistics about the dataset"""
        stats = {
            'Total Tracks': len(self.tracks_df),
            'Unique Artists': self.tracks_df['artist'].nunique(),
            'Unique Albums': self.tracks_df['album'].nunique(),
            'Tracks by Mood': self.tracks_df['mood'].value_counts().to_dict(),
            'Average Popularity': round(self.tracks_df['popularity'].mean(), 2)
        }
        return stats

# Create the recommender
recommender = MoodBasedRecommender(tracks_df)

# Display stats
print("✓ Recommendation system ready!")
print("\nDataset Statistics:")
stats = recommender.get_mood_stats()
for key, value in stats.items():
    print(f"  • {key}: {value}")

✓ Recommendation system ready!

Dataset Statistics:
  • Total Tracks: 385
  • Unique Artists: 295
  • Unique Albums: 353
  • Tracks by Mood: {'Calm': 50, 'Energetic': 50, 'Party': 50, 'Romantic': 50, 'Happy': 49, 'Sad': 49, 'Motivated': 45, 'Focus': 42}
  • Average Popularity: 48.53


In [16]:
# Test the recommendation system for each mood
print("="*60)
print("MOOD-BASED MUSIC RECOMMENDATIONS")
print("="*60)

for mood in ['Happy', 'Sad', 'Energetic', 'Calm']:
    print(f"\n{'─'*60}")
    print(f"Top recommendations for {mood.upper()} mood:")
    print(f"{'─'*60}")
    recommendations = recommender.recommend_by_mood(mood, n_recommendations=5)
    display(recommendations)

MOOD-BASED MUSIC RECOMMENDATIONS

────────────────────────────────────────────────────────────
Top recommendations for HAPPY mood:
────────────────────────────────────────────────────────────


,name,artist,album,popularity
0,Happy Nation,Ace of Base,Happy Nation,74
43,Afghan Jalebi (Film Version),Pritam,Phantom,70
9,Aashiq Tera,Sohail Sen,Happy Bhag Jayegi (Original Motion Picture Sou...,69
30,Joyful,Calmly,Daydreams,67
35,chess - slowed,joyful,chess,67



────────────────────────────────────────────────────────────
Top recommendations for SAD mood:
────────────────────────────────────────────────────────────


,name,artist,album,popularity
67,Call Out My Name,The Weeknd,"My Dear Melancholy,",85
60,On Melancholy Hill,Gorillaz,Plastic Beach,84
66,I Was Never There,The Weeknd,"My Dear Melancholy,",83
90,Lonely,Akon,Trouble Deluxe Edition,79
87,Heartbreak Anniversary,GIVĒON,TAKE TIME,78



────────────────────────────────────────────────────────────
Top recommendations for ENERGETIC mood:
────────────────────────────────────────────────────────────


,name,artist,album,popularity
110,Pumped Up Kicks,Foster The People,Torches,84
111,Pump It,Black Eyed Peas,Monkey Business,81
100,Work Out,J. Cole,Cole World: The Sideline Story,80
132,POWER,Kanye West,My Beautiful Dark Twisted Fantasy,78
127,Excuses,AP Dhillon,Excuses,73



────────────────────────────────────────────────────────────
Top recommendations for CALM mood:
────────────────────────────────────────────────────────────


,name,artist,album,popularity
196,snowfall,Øneheart,snowfall,82
163,Runaway,AURORA,All My Demons Greeting Me As A Friend (Deluxe),77
167,Peaceful Sleep,Lullaby Time,Baby Sleep Music,75
190,Focus On Noise,Hovar,Lean Noise,75
155,Relaxing Brown Noise 256 Hz Smooth Q,Jane Color,Relaxing Brown Noise 256 Hz,70


## How to Use the Recommendation System

You can use the following methods:

1. **Get recommendations by mood:**
   ```python
   recommender.recommend_by_mood('Happy', n_recommendations=10)
   ```

2. **Find similar tracks:**
   ```python
   recommender.recommend_similar('track_name', n_recommendations=5)
   ```

3. **Get dataset statistics:**
   ```python
   recommender.get_mood_stats()
   ```

In [17]:
# Example: Get recommendations based on user mood input
print("Try the recommendation system:")
print("="*60)

# Example 1: Recommend by mood
user_mood = 'Calm'  # Change this to: 'Happy', 'Sad', 'Energetic', or 'Calm'
print(f"\n🎵 Top {user_mood} tracks:")
recommendations = recommender.recommend_by_mood(user_mood, n_recommendations=5)
display(recommendations)

# Example 2: Find similar tracks
print(f"\n🔍 Finding tracks similar to a popular song...")
# Get a random popular track from the dataset
sample_track = tracks_df.nlargest(1, 'popularity').iloc[0]
print(f"Based on: '{sample_track['name']}' by {sample_track['artist']}")
similar_tracks = recommender.recommend_similar(sample_track['name'], n_recommendations=5)
display(similar_tracks)

Try the recommendation system:

🎵 Top Calm tracks:


,name,artist,album,popularity
196,snowfall,Øneheart,snowfall,82
163,Runaway,AURORA,All My Demons Greeting Me As A Friend (Deluxe),77
167,Peaceful Sleep,Lullaby Time,Baby Sleep Music,75
190,Focus On Noise,Hovar,Lean Noise,75
155,Relaxing Brown Noise 256 Hz Smooth Q,Jane Color,Relaxing Brown Noise 256 Hz,70



🔍 Finding tracks similar to a popular song...
Based on: 'Call Out My Name' by The Weeknd


,name,artist,popularity,mood
60,On Melancholy Hill,Gorillaz,84,Sad
66,I Was Never There,The Weeknd,83,Sad
90,Lonely,Akon,79,Sad
87,Heartbreak Anniversary,GIVĒON,78,Sad
95,Lonely Road (with Jelly Roll),mgk,74,Sad


In [22]:
# 🎵 TEST THE MOOD-BASED RECOMMENDATION SYSTEM 🎵
# Change the mood below to get different recommendations

# Available moods: 'Happy', 'Sad', 'Energetic', 'Calm', 'Romantic', 'Motivated', 'Party', 'Focus'
input_mood = 'Romantic'  # ← Change this to test different moods
num_songs = 10  # ← Change this to get more or fewer songs

print("="*70)
print(f"🎵 GETTING {num_songs} SONG RECOMMENDATIONS FOR '{input_mood.upper()}' MOOD")
print("="*70)

# Get recommendations
result = recommender.recommend_by_mood(input_mood, n_recommendations=num_songs)

# Display results
if isinstance(result, str):
    print(f"\n❌ {result}")
    print(f"\n📋 Available moods:")
    for mood in sorted(tracks_df['mood'].unique()):
        count = len(tracks_df[tracks_df['mood'] == mood])
        print(f"   • {mood} ({count} tracks)")
else:
    print(f"\n✓ Found {len(result)} songs for {input_mood} mood:\n")
    display(result)
    
    print(f"\n📊 Summary:")
    print(f"   • Average popularity: {result['popularity'].mean():.1f}")
    print(f"   • Most popular: '{result.iloc[0]['name']}' by {result.iloc[0]['artist']}")
    print(f"   • Unique artists: {result['artist'].nunique()}")

🎵 GETTING 10 SONG RECOMMENDATIONS FOR 'ROMANTIC' MOOD

✓ Found 10 songs for Romantic mood:



,name,artist,album,popularity
200,Romantic Homicide,d4vd,Romantic Homicide,87
230,Passionfruit,Drake,More Life,85
210,Love Me Not,Ravyn Lenae,Love Me Not / Love Is Blind,84
217,Lover,Taylor Swift,Lover,84
206,Apna Bana Le,Sachin-Jigar,Bhediya (Original Motion Picture Soundtrack),78
208,"Barbaad (From ""Saiyaara"")",The Rish,"Barbaad (From ""Saiyaara"")",78
211,Love Story (Taylor’s Version),Taylor Swift,Love Story (Taylor’s Version),77
213,Love Story,Indila,Mini World,77
214,"Love Me Like You Do - From ""Fifty Shades Of Grey""",Ellie Goulding,Delirium (Deluxe),77
240,Valentine,Laufey,Everything I Know About Love,77



📊 Summary:
   • Average popularity: 80.4
   • Most popular: 'Romantic Homicide' by d4vd
   • Unique artists: 9
